In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Some Helper Functions

In [2]:
'''a simple one_hot_encoder which encode the numbers from 0 to 9 in vector
for example: if y = 3  then one hot vector is [0,0,0,1,0,0,0,0,0,0]'''

def one_hot_encode(y, num_classes=10):
    m = y.shape[0]
    one_hot = np.zeros((m, num_classes))
    one_hot[np.arange(m), y] = 1
    return one_hot

In [3]:
'''ReLU (Rectified Linear Unit) its a simple to introduce non-linearity means it just convert y = x if x>0 and y = 0 if x < 0 '''

def relu(Z):
    return np.maximum(0,Z)   #if Z<0 then return 0 otherwise return Z 

In [ ]:
def softmax(Z):
    pass

In [4]:
def cross_entropy_loss(Y_pred, Y_true):
    pass

## Neural Network Class: Neural_Network